In [1]:
from astropy.cosmology import FlatLambdaCDM
import astropy.units as u
import numpy as np
import matplotlib.pyplot as plt
from astropy.io import ascii
from astropy import constants as const

## Trying to Replicate SFR values listed in Terrazas et al 2017 
https://ui.adsabs.harvard.edu/abs/2017ApJ...844..170T/abstract

Following methodology presented in the paper as I understand it 

In [ ]:
#Setting the model so that we can use it to calculate the luminosity distance 
    # H0=Hubble constant at z = 0. If a float, must be in [km/sec/Mpc].
    # Om0 = Omega matter: density of non-relativistic matter in units of the critical density at z=0.
    # Tcmb0 = Temperature of the CMB z=0. If a float, must be in [K]. Default: 0 [K].
cosmo = FlatLambdaCDM(H0=70 * u.km / u.s / u.Mpc, Tcmb0=2.725 * u.K, Om0=0.3)

c = const.c
c

From reading both the Terrazas et al 2017 and the Terrazas et al 2016b paper (https://ui.adsabs.harvard.edu/abs/2016ApJ...830L..12T/abstract), I believe this is the procees to obtain SFRs in the same manner: 
- Use FIR flux from IRAS (Bell et al 2003 used 60 and 100 micron to estimate the FIR flux) 
- Estimate the TIR flux via TIR = 2*FIR (sum of 60 and 100 micron)
- Use Equation 12 of Kennicutt and Evans (2012) {https://www.annualreviews.org/doi/pdf/10.1146/annurev-astro-081811-125610 page 27}: 
    - logSFR(Mdot/yr) = logLTIR(erg/s) - logCx where logCx is 43.41 

As a test to make sure that I was replicating the correct methodology, I used NGC 4258 as my test galaxy where I am hoping to get a logSFR of -0.080

In [2]:
#First I pull the IRAS flux density values (Jy) for 60 and 100 microns from NED 
    #https://ned.ipac.caltech.edu/byname?objname=ngc+4258&hconst=67.8&omegam=0.308&omegav=0.692&wmap=4&corr_z=1

#flux density (wrote out Janskys)
fd_60 = 21.6  
print('flux density for 60 microns [Jy]:', fd_60)

fd_100 = 78.39 
print('flux density for 100 microns [Jy]:', fd_100)

flux density for 60 microns [Jy]: 21.6
flux density for 100 microns [Jy]: 78.39


In [ ]:
#Because these are flux densities I thought the first step would be to get them into fluxes which is the process described here 

#Frequency observations taken at 60 microns
freq_60 = 5e12 * u.Hz  
print('Freqency of 60 microns:', freq_60)

#Frequency observations taken at 100 microns 
freq_100 = 3e12 * u.Hz 
print('Freqency of 100 microns:', freq_100)

################################################

#Calculating the wavelength from the freq and putting into angstroms 

#60 microns
wav_60 = c/freq_60
wav_60 = wav_60.to(u.Angstrom)
print("Wavlength in Anstroms:", wav_60)

#100 microns
wav_100 = c/freq_100
wav_100 = wav_100.to(u.Angstrom)
print("Wavlength in Anstroms:", wav_100)

In [ ]:
# I then use a conversion equation to get a flux density but in units that includes angstroms so that I can divide out
    #the angstrom dependence later to get the flux 

#Formula comes from here: https://www.stsci.edu/~strolger/docs/UNITS.txt
    # (Y erg/cm^2/s/A) = 2.99792458E-05 *(X1 Jy) / (X2 A)^2
    #Where X1 is your flux density in Jy and X2 is the frequency in Hz of the bandpass converted to a wavelength in Angstroms.


fdA_60 = (2.99792458e-5 * ((fd_60)/(599584.9159999999)**2)) * (u.erg/(u.cm**2 * u.s *u.Angstrom))
print(fdA_60)

fdA_100 = (2.99792458e-5 * ((fd_100)/(999308.1933333331)**2)) * (u.erg/(u.cm**2 * u.s *u.Angstrom))
print(fdA_100)

In [ ]:
#To get it in units of FLUX I have to mutiply by the wavelength

FIR_60 = fdA_60 * wav_60 
print('FLUX of 60',FIR_60)

FIR_100 = fdA_100 * wav_100
print('FLUX of 100', FIR_100)

In [ ]:
#Now I need to get the sum flux or FIR and then the TIR using the equation presented in Terrazas et al 2016b 

#From the paper it says that the summed the flux for IRAS 
FIR = FIR_60 + FIR_100

TIR = 2 * FIR
print('Total Infrared Flux:', TIR)

In [ ]:
#I figured that I would need to use the luminosity distance rather than just the normal distance so I calculated that using cosmo 

d_lum = cosmo.luminosity_distance(0.001494) #using the redshift and then putting into cm 
d_lum = d_lum.to(u.cm)

print("Luminosity Distance:", d_lum)


In [ ]:
#Now I can calculate the Luminosity which should come out to be in units of (u.erg/u.s) which is what is needed for the 
    #Kennicutt and Evans equation 

L = 4*np.pi*TIR*d_lum**2 #* (u.erg/u.s)
print(L)

In [ ]:
#Putting that calculated luminosity into the equation 

log10SFR_TIR = np.log10(3.3697777532746364e+43) - 43.41
print('My found logSFR for NGC 4258:', log10SFR_TIR)

#Seeing what the not logged version is 
print('Unlogged SFR:', 10**0.117)

In [ ]:
#Comparing to what I was supposed to get

print('logSFR (Mdot/yr): -0.08')
print('Unlogged SFR:', 10**-0.08)

### Since there was this disconnect, Jess and I went to Bell et al 2003 and used the equations presented there to see if they were similar 
https://ui.adsabs.harvard.edu/abs/2003ApJ...586..794B/abstract

In [ ]:
#flux density (wrote out Janskys)
fd_60 = 21.6 *  u.Jy
print('flux density for 60 microns:', fd_60)

fd_100 = 78.39 * u.Jy 
print('flux density for 100 microns:', fd_100)

In [ ]:
fd_60_watt = fd_60.to(u.Watt/(u.m**2 * u.Hz))
print('Flux density of 60 microns in Watts:', fd_60_watt) 

fd_100_watt = fd_100.to(u.Watt/(u.m**2 * u.Hz))
print('Flux density of 60 microns in Watts:', fd_100_watt) 

In [ ]:
#To make the units work out need to put the luminosity distance in untis of meters 

d_lum = cosmo.luminosity_distance(0.001494) #using the redshift and then putting into cm 
d_lum = d_lum.to(u.m)
d_lum

In [ ]:
#Now want to calculate the Luminosity and to make sure that the units work out, need to multiply the fd by the frequency 

L_60 = ((fd_60_watt)*freq_60)*4*np.pi* d_lum**2
L_100 = ((fd_100_watt)*freq_100)*4*np.pi* d_lum**2

print('Luminoity of 60 microns:', L_60)
print('Luminosity of 100 microns', L_100)

In [ ]:
#Checking to see if these values of luminosity are similar to the presented values in Bell et al 2003 (Table A1) 
    #Note, in this paper it looks like TIR refers to luminosities while in the Terrazas papers they use TIR to mean flux 
    
print(np.log10(5.302561374153637e+35))
print(np.log10(1.1546327392219544e+36))

In [ ]:
#The values are similar to the ones presented in the table so we move on to summing th luminosities for LTIR 

TIR = (L_100 + L_60)
TIR

In [ ]:
#Need to get the Solar LTIR as needed for using equation 4 in Bell et al 2003 

L_TIR_solar = TIR/(3.9e26 *u.Watt)
L_TIR_solar

In [ ]:
#Now calculating the SFR using said equation 4 from Bell et al 2003 
SFR = (1.72e-10*L_TIR_solar) * (1+ np.sqrt(10**9/L_TIR_solar))
SFR

In [ ]:
np.log10(SFR)

So these are closer to my values and there is still a disconnect with the value for the published SFR value for NGC4258

Is there maybe an upper limit ratio that I am missing or maybe I am not understanding the methodology correctly? Any help/advice you could provide would be greatly appreciated! Thanks! 

# Help from Bryan 

Need to make sure that we are taking into account the correct distance 

Also, you're setting FIR = f_60 + f_100, when the relation should be a little different. Instead of adding the two fluxes to get the total FIR, you'll want to approximate the area under the SED (i.e., the total flux) given the value of the SED at 2 particular wavelengths (in this case 60+100 micron) to get the total FIR flux. This of course assumes a shape for the SED, but it's a rough approximation! My PhD advisor, Eric Bell, has already done that approximation and I use that relation from Bell et al. (2003), pg 17, eqn A1. Here's my code for converting fluxes to SFR:

In [6]:
#First I pull the IRAS flux density values (Jy) for 60 and 100 microns from NED 
    #https://ned.ipac.caltech.edu/byname?objname=ngc+4258&hconst=67.8&omegam=0.308&omegav=0.692&wmap=4&corr_z=1

#flux density (wrote out Janskys)
fd_60 = 21.6  * u.Jy
print('flux density for 60 microns [Jy]:', fd_60)

fd_100 = 78.39 * u.Jy
print('flux density for 100 microns [Jy]:', fd_100)

flux density for 60 microns [Jy]: 21.6 Jy
flux density for 100 microns [Jy]: 78.39 Jy


In [ ]:
# I want to make arrays with all the information for Terrazas 2017 galaxies 



In [7]:
#distance that we really want to use

d_Mpc = 7.27 * u.Mpc #Mpc
print(d_Mpc)

d_pc = 7.27e6 * u.pc 
print(d_pc)

d_meters = d_pc.to(u.m)
print(d_meters)

7.27 Mpc
7270000.0 pc
2.243287601744224e+23 m


In [8]:
#Bryan's Code

def L60_L100_toSFR(f_60, f_100, d):
    #From Bell+03: Equation A1
    FIR = (1.26*10**(-14)) * ((2.58*f_60) + f_100) # comes out in W m-2 (f_* should be in Jy)
    TIR = 2.0*FIR #this we know from Terrazas 2016b
    
    #From Kennicutt & Evans 2012:
    #The 10^7 is the unit conversion factor from watts to erg/s.
    L_TIR = (4.0*np.pi*TIR * d**2) * 10**7 #d is in meters (OG code had lots of conversions to get Mpc to m)
    L_TIR = L_TIR.astype(float)
    
    logC_TIR = 43.41 #constant from K&E 2012 equation 12
    SFRnolog = 10**(np.log10(L_TIR) - logC_TIR) #Msun/yr
    sfrlog = (np.log10(L_TIR) - logC_TIR) 
    
    return SFRnolog, sfrlog

L60_L100_toSFR(fd_60.value, fd_100.value, d_meters.value)

(0.83150982561298, -0.08013261453311316)